# **MODELING and EVALUATION**

## **Business Requirement**
- Address **Business Requirement 2**: Develop a classification model to detect **powdery mildew** on cherry leaves.
- Automate disease detection to **save time** and **improve efficiency** in plantation management.
- Compare different **machine learning models** to determine the **most efficient and accurate** solution.

---

## **Objectives**
1. **Baseline CNN Model**: Implement a **simplified CNN** to establish a reference point.
2. **Transfer Learning with MobileNetV2**: Use **pre-trained MobileNetV2** to leverage feature extraction from ImageNet.
3. **Optimized Hyperparameter Tuning**: Perform a **lightweight** tuning process to improve performance **without high computation costs**.
4. **Model Evaluation & Explainability**:  
   - Compare **CNN vs. MobileNetV2** performance (baseline & tuned).  
   - Visualize **model focus areas** using **Grad-CAM**.  
5. **Final Model Selection**: Choose the best model for **deployment** based on **accuracy, efficiency, and business requirements**.

---

## **Inputs**
### **Dataset**
- **Train Data** → `inputs/mildew_dataset/cherry-leaves/train`
- **Validation Data** → `inputs/mildew_dataset/cherry-leaves/validation`
- **Test Data** → `inputs/mildew_dataset/cherry-leaves/test`

### **Precomputed Features (from Data Visualization Notebook)**
- **Image Shape Standardization** → **128x128x3** for consistency across models.
- **Class Distribution Analysis** → Ensures balanced dataset splits.
- **Feature Space Visualization** → **PCA confirms class separability**.
- **Pixel Intensity Distribution** → Confirms brightness variations relevant for classification.

---

## **Outputs**
### **Data Processing & Visualization**
- **Dataset Distribution Plot** → Confirms class balance across training, validation, and test sets.
- **Data Augmentation Visualization** → Shows the impact of transformations applied to training data.

### **Model Training & Optimization**
1. **Simplified CNN Model** → Establishes baseline performance.
2. **MobileNetV2 Transfer Learning** → Leverages pre-trained knowledge for improved classification.
3. **Hyperparameter Tuning** → Fine-tunes **learning rate, dropout, and L2 regularization** while keeping computation light.
4. **Best Model Selection** → Based on **test accuracy and generalization ability**.
5. **Saved Trained Models** → Ready for **Streamlit integration** and **deployment**.

### **Model Evaluation & Explainability**
- **Training Progress Visualization** → Learning curves for loss & accuracy trends.
- **Confusion Matrix** → Static visualizations for **train, validation, and test sets**.
- **Classification Report** → Detailed **precision, recall, and F1-score** breakdown.
- **Grad-CAM Heatmap** → Highlights **which areas of the image** were most important in classification.

### **Final Model Selection**
- **Compare CNN vs. MobileNetV2 (Base & Tuned)**
- **Choose the best model based on performance, efficiency, and deployment feasibility**
- **Deploy the final model for real-world use in a Streamlit application**

---

## **Additional Comments**
- **Business Impact**: A reliable model reduces **manual inspection time** and improves **plantation monitoring efficiency**.
- **Data-Driven Enhancements**: Model optimizations are based on **insights from data preprocessing and visualization**.
- **Deployment-Ready**: The best-performing model is **optimized for integration into a Streamlit web app**.

---

## **Setup & Data Preparation**

---

### **Import Necessary Packages**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

### **Set Working Directory & File Paths**

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspaces/mildew-detection-app')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

**Set Input Directories**

In [ ]:
# Set train, validation and test paths
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'


**Set Output Directory**

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### **Load Dataset & Labels**

**Set Labels**

In [ ]:
# Set the lables for the images
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

**Load Precomputed Image Shape from Data Visualization Notebook**

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

### **Count and Visualize Dataset Distribution**

**Count Images in Each Dataset Split**

In [ ]:
# Create an empty dictionary
data = {
    'Set': [],
    'Label': [],
    'Frequency': []
}

# Define dataset folders
folders = ['train', 'validation', 'test']

# Loop through each dataset split and count images
for folder in folders:
    for label in labels:
        path = os.path.join(my_data_dir, folder, label)
        num_images = len(os.listdir(path)) if os.path.exists(path) else 0  # Handle missing directories
        data['Set'].append(folder)
        data['Label'].append(label)
        data['Frequency'].append(num_images)
        print(f" {folder}/{label}: {num_images} images")

# Convert dictionary to DataFrame
df_freq = pd.DataFrame(data)

# ** Bar Chart - Image Distribution**
print(f"Bar Chart: Image distribution across dataset splits.")
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.title("Image Distribution in Dataset")
plt.xlabel("Dataset Split")
plt.ylabel("Number of Images")
plt.savefig(f'{file_path}/labels_distribution.png', bbox_inches='tight', dpi=150)
plt.show()

# ** Pie Chart - Class Distribution**
print(f"Pie Chart: Overall class distribution (Healthy vs Infected).")
plt.figure(figsize=(5, 5))
label_distribution = df_freq.groupby("Label")["Frequency"].sum()
plt.pie(label_distribution, labels=label_distribution.index, autopct='%1.1f%%', startangle=90, colors=["#1f77b4", "#ff7f0e"])
plt.title("Overall Class Distribution (Healthy vs Infected)")
plt.savefig(f'{file_path}/labels_pie_chart.png', bbox_inches='tight', dpi=150)
plt.show()

---

## **Implement Data Augmentation**

---

### **Define Augmentation Techniques**

Rotation, width/height shift, zoom, shear, horizontal/vertical flip, rescaling.

In [ ]:
# Import TensorFlow/Keras ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### **Augment Training, Validation, and Test Sets**

**Initialize ImageDataGenerator for Data Augmentation**

In [ ]:
# Define Augmentation for Training Set
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

**Augment Training Image Dataset**

In [ ]:
batch_size = 20  # Set batch size
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

train_set.class_indices

**Augment Validation Image Dataset**

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

**Augment Test Image Dataset**

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

### **Visualization of Augmented Images**

**Plot Augmented Training Image**

In [ ]:
for _ in range(3):
    img, label = next(train_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

**Plot Augmented Validation and Test Images**

In [ ]:
for _ in range(3):
    img, label = next(validation_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(3):
    img, label = next(test_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

**Save Class Indices**

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

### **Compare Multiple Augmented Images in a Grid**

In [ ]:
def plot_augmented_images_grid(data_generator, num_images=10):
    """Displays a grid of augmented images to visualize transformation effects."""
    img_batch, label_batch = next(data_generator)

    fig, axes = plt.subplots(2, num_images // 2, figsize=(15, 6))
    
    for i in range(num_images):
        ax = axes[i // (num_images // 2), i % (num_images // 2)]
        ax.imshow(img_batch[i])
        ax.axis("off")

    plt.suptitle("Augmented Image Variations (Training Set)")
    plt.show()

# Display the augmented image grid
plot_augmented_images_grid(train_set)

---

## **Model Creation**

---

**Import Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### **Base Convolutional Neural Network (CNN)**

This **lightweight CNN** is designed as a baseline to classify cherry leaves as **Healthy** or **Infected with Powdery Mildew** before applying transfer learning.

- **Three Convolutional Layers** → Extract key features efficiently.
- **Batch Normalization** → Stabilizes training and improves convergence.
- **L2 Regularization (`0.0001`)** → Prevents overfitting without excessive constraint.
- **Dropout (`0.3`)** → Enhances generalization by reducing reliance on specific neurons.
- **Max Pooling (`2x2`)** → Reduces computational cost while keeping essential patterns.
- **Adam Optimizer (`0.001` learning rate)** → Balances speed and stability.
- **Sigmoid Activation (Binary Classification)** → Outputs probability for "Infected" class.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


def create_base_cnn():
    """
    Creates a simplified CNN model for binary classification.

    Improvements:
    - Fewer convolutional layers (3 instead of 4) for efficiency
    - Uses BatchNormalization after each Conv layer for stability
    - Lower dropout rate (0.3 instead of 0.5) to balance regularization
    - Learning rate increased to 0.001 for faster convergence
    - Less aggressive L2 regularization to reduce training cost

    Returns:
        model: A compiled Keras CNN model
    """

    model = Sequential(
        [
            # First convolutional block
            Conv2D(
                filters=32,
                kernel_size=(3, 3),
                activation="relu",
                input_shape=image_shape,
                kernel_regularizer=l2(0.0001),
            ),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2)),
            # Second convolutional block
            Conv2D(
                filters=64,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.0001),
            ),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2)),
            # Third convolutional block
            Conv2D(
                filters=128,
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(0.0001),
            ),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2)),
            # Flatten layer to convert feature maps into a single vector
            Flatten(),
            # Fully connected layer with regularization
            Dense(128, activation="relu", kernel_regularizer=l2(0.0001)),
            Dropout(0.3),  # Lower dropout for better training
            # Output layer for binary classification
            Dense(1, activation="sigmoid"),
        ]
    )

    # Compile Model
    model.compile(
        optimizer=Adam(learning_rate=0.001),  # Slightly higher learning rate
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model


# Instantiate the CNN model
model_cnn = create_base_cnn()


**Model Summary**

In [ ]:
# Print Model Summary
model_cnn.summary()

### **Train CNN Model with Early Stopping**

**Implement EarlyStopping to prevent overfitting**

In [ ]:
# Set EarlyStopping with a lower patience for faster convergence
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

**Train the CNN model for 20 epochs**

In [ ]:
# Create the CNN model
model = create_base_cnn()

# Store the training history
history = model.fit(
    train_set,
    epochs=20,
    steps_per_epoch=len(train_set.classes) // batch_size,
    validation_data=validation_set,
    callbacks=[early_stop],
    verbose=1
)

**Load and Save the CNN Model**

In [ ]:
# Save the model with the original file name
model_cnn.save("outputs/v1/mildew_detector_cnn.keras")
print("Model saved successfully as 'mildew_detector_cnn.keras'")

---

## **Model Performance & Evaluation**

---

### **Evaluate CNN on the Test Set**

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_set)

# Print evaluation results
print(f" Test Accuracy: {test_accuracy:.4f}")
print(f" Test Loss: {test_loss:.4f}")

### **Plot Learning Curves**

In [ ]:
# Convert training history to DataFrame
losses = pd.DataFrame(history.history)

# Set Seaborn style
sns.set_style("whitegrid")

# Plot Loss Curve
plt.figure(figsize=(6, 4))
losses[["loss", "val_loss"]].plot(style=".-")
plt.title("Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["Training Loss", "Validation Loss"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_losses.png", bbox_inches="tight", dpi=150)
plt.show()

# Plot Accuracy Curve
plt.figure(figsize=(6, 4))
losses[["accuracy", "val_accuracy"]].plot(style=".-")
plt.title("Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Training Accuracy", "Validation Accuracy"])
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{file_path}/model_training_acc.png", bbox_inches="tight", dpi=150)
plt.show()

### **Confusion Matrix & Classification Report**


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Create output directory
output_dir = "outputs/v1"
os.makedirs(output_dir, exist_ok=True)


def generate_confusion_matrix(y_true, y_pred, label_map, set_name):
    """
    Generates and saves a static confusion matrix.
    """
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=label_map, columns=label_map)

    # Plot confusion matrix
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", linewidths=0.5)
    plt.title(f"Confusion Matrix - {set_name} Set")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()

    # Save the confusion matrix
    save_path = os.path.join(output_dir, f"confusion_matrix_{set_name.lower()}.png")
    plt.savefig(save_path, dpi=150)
    plt.close()  # Prevent multiple displays
    print(f"Confusion Matrix saved: {save_path}")

    return cm


def generate_classification_report(y_true, y_pred, label_map, set_name):
    """
    Generates and prints a classification report.
    """
    report = classification_report(y_true, y_pred, target_names=label_map)
    print(f"\n--- Classification Report: {set_name} Set ---\n")
    print(report)

    # Save report as a text file
    report_path = os.path.join(
        output_dir, f"classification_report_{set_name.lower()}.txt"
    )
    with open(report_path, "w") as f:
        f.write(report)
    print(f"Classification report saved: {report_path}")

    return report


def evaluate_model(generator, model, label_map, set_name, threshold=0.5):
    """
    Evaluates model performance by generating a confusion matrix and classification report.
    """
    y_true = generator.classes
    y_pred_probs = model.predict(generator)
    y_pred = (y_pred_probs > threshold).astype(int).flatten()

    print(f"\n#### {set_name} Set Evaluation ####\n")

    cm = generate_confusion_matrix(y_true, y_pred, label_map, set_name)
    report = generate_classification_report(y_true, y_pred, label_map, set_name)

    return cm, report



In [ ]:
# Get label names from training set
label_map = list(train_set.class_indices.keys())

# Evaluate on Train, Validation, and Test sets
train_cm, train_report = evaluate_model(train_set, model, label_map, "Train")
val_cm, val_report = evaluate_model(validation_set, model, label_map, "Validation")
test_cm, test_report = evaluate_model(test_set, model, label_map, "Test")

### **McNemar’s Test for Model Comparison**

McNemar’s test is used to compare the performance of two classification models on the **same dataset** and determine if their differences in predictions are **statistically significant**.

**Why McNemar’s Test?**
- **Beyond Accuracy** → Instead of just comparing accuracy scores, it evaluates whether **one model consistently outperforms another** in misclassification cases.
- **Paired Comparison** → Since both models predict the same test set, McNemar’s test measures if **their disagreements are random or meaningful**.
- **Significance Test** → If the **p-value < 0.05**, the models have a statistically significant difference in performance.

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from tensorflow.keras.models import load_model

output_dir = "outputs/v1"
os.makedirs(output_dir, exist_ok=True)


### McNemar’s Test for Model Comparison
def mcnemar_test(test_set, model_1, model_2):
    """
    Perform McNemar's test to compare two classification models.
    Checks if there is a statistically significant difference between them.
    """

    print("\n### Running McNemar's Test ###")

    # Extract ground-truth labels
    y_true = test_set.classes

    # Get predictions from both models
    y_pred_model_1 = (model_1.predict(test_set) > 0.5).astype(int).flatten()
    y_pred_model_2 = (model_2.predict(test_set) > 0.5).astype(int).flatten()

    # Compute contingency table
    a = np.sum((y_pred_model_1 == y_true) & (y_pred_model_2 == y_true))  # Both correct
    b = np.sum(
        (y_pred_model_1 == y_true) & (y_pred_model_2 != y_true)
    )  # Model 1 correct, Model 2 wrong
    c = np.sum(
        (y_pred_model_1 != y_true) & (y_pred_model_2 == y_true)
    )  # Model 1 wrong, Model 2 correct
    d = np.sum((y_pred_model_1 != y_true) & (y_pred_model_2 != y_true))  # Both wrong

    contingency_table = np.array([[a, b], [c, d]])
    print("McNemar's Contingency Table:\n", contingency_table)

    # Perform McNemar's test
    result = mcnemar(contingency_table, exact=False)
    print(f"Chi-square statistic: {result.statistic:.4f}")
    print(f"P-value: {result.pvalue:.6f}")

    # Interpretation
    if result.pvalue < 0.05:
        print("There is a significant difference between the models.")
    else:
        print("No significant difference between the models.")

    return result.pvalue

### **Model Evaluation**

**Load Saved Model**

In [ ]:
model_path = "outputs/v1/mildew_detector_mobilenet.keras"  
model = load_model(model_path)
print("\nModel loaded successfully!")

**Evaluate Model Performance**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_set, verbose=0)

**Save Evaluation Pickle**

In [ ]:
evaluation = {"test_loss": test_loss, "test_accuracy": test_accuracy}
joblib.dump(value=evaluation, filename=f"{output_dir}/evaluation.pkl")
print("\nModel evaluation results saved!")

### **Check Business Case Requirement**

In [ ]:
required_accuracy = 0.90  # Business goal threshold

print("\n### Performance Evaluation Summary ###")
print(f"Required Accuracy: {required_accuracy * 100:.2f}%")
print(f"Model Achieved Accuracy: {test_accuracy * 100:.2f}%\n")

if test_accuracy >= required_accuracy:
    print("The model meets the business performance requirement.")
else:
    print("The model does NOT meet the business requirement. Consider further tuning.")

---

## **Overfitting Analysis** 

---

**Analyze overfitting by comparing training vs. validation accuracy & loss**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert training history to DataFrame
losses = pd.DataFrame(history.history)

# Set style for plots
sns.set_style("whitegrid")

# Plot Training & Validation Loss
plt.figure(figsize=(8, 4))
plt.plot(losses["loss"], label="Training Loss", marker="o")
plt.plot(losses["val_loss"], label="Validation Loss", marker="o")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 4))
plt.plot(losses["accuracy"], label="Training Accuracy", marker="o")
plt.plot(losses["val_accuracy"], label="Validation Accuracy", marker="o")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Identify Overfitting
train_acc = losses["accuracy"].iloc[-1]
val_acc = losses["val_accuracy"].iloc[-1]
train_loss = losses["loss"].iloc[-1]
val_loss = losses["val_loss"].iloc[-1]

print("\n### Overfitting Analysis ###")
print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Final Training Loss: {train_loss:.4f}")
print(f"Final Validation Loss: {val_loss:.4f}")

# Simple Overfitting Check
if train_acc - val_acc > 0.05:  # If training accuracy is much higher than validation
    print("\nWarning: Possible Overfitting Detected!")
    print(
        "Consider using Dropout, L2 Regularization, or reducing the number of layers."
    )
else:
    print("\nNo significant overfitting detected. The model generalizes well.")

---

## **Hyperparameter Tuning with Keras Tuner** 

---

Optimize the CNN model using Keras Tuner to improve performance while preventing overfitting.

**Import Libraries**

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

### **Define the Hyperparameter Search Space**

Optimize the following parameters:
- Number of Filters → [32, 64]
- L2 Regularization (Weight Decay) → [0.0001, 0.001]
- Dropout Rate → [0.2, 0.5]
- Learning Rate → [0.0001, 0.0005]

In [ ]:
# Function to define CNN model with hyperparameter tuning
def build_model(hp):
    """
    Define a CNN model with hyperparameter tuning using Keras Tuner.

    Parameters:
    - hp: Keras Tuner search space

    Returns:
    - Compiled Keras model
    """
    model = Sequential(
        [
            Conv2D(
                filters=hp.Choice("num_filters", values=[32, 64]),
                kernel_size=(3, 3),
                activation="relu",
                input_shape=image_shape,
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(
                filters=hp.Choice("num_filters", values=[32, 64]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=l2(hp.Choice("l2_reg", values=[0.0001, 0.001])),
            ),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(128, activation="relu"),
            Dropout(hp.Choice("dropout_rate", values=[0.2, 0.5])),
            Dense(1, activation="sigmoid"),
        ]
    )

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Choice("learning_rate", values=[0.0001, 0.0005])
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

### **Run Hyperparameter Search**

Search for the best hyperparameters using Keras Tuner’s RandomSearch.

In [ ]:
# Define the tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",  # Optimize for highest validation accuracy
    max_trials=5,  # Limits the number of model variations
    executions_per_trial=1,  # Runs each model once
    directory="keras_tuner_results",
    project_name="cnn_tuning",
)

# Run hyperparameter search
tuner.search(
    train_set,
    validation_data=validation_set,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)],
)

###  **Retrieve Best Hyperparameters & Train Final Model**

In [ ]:
# Retrieve best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n### Best Hyperparameters Found ###")
print(f"Filters: {best_hps.get('num_filters')}")
print(f"L2 Regularization: {best_hps.get('l2_reg')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")

# Train the best model
best_model = tuner.hypermodel.build(best_hps)
history_best = best_model.fit(
    train_set,
    validation_data=validation_set,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)],
    verbose=1,
)

# Save the best model
best_model.save("outputs/v1/mildew_detector_cnn_tuned.keras")
print("\nBest tuned CNN model saved successfully!")

### **Evaluate the Tuned Model**

In [ ]:
best_model = tf.keras.models.load_model("outputs/v1/mildew_detector_cnn_tuned.keras")
test_loss, test_accuracy = best_model.evaluate(test_set)

print("\n### Tuned Model Evaluation ###")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

### **Compare Tuned Model vs. Original CNN**

Compare the original CNN with the tuned CNN to check if hyperparameter tuning improved the performance.

### **Load Both Models**

In [ ]:
from tensorflow.keras.models import load_model

# Load the original CNN model
original_model = load_model("outputs/v1/mildew_detector_cnn.keras")

# Load the best tuned CNN model
tuned_model = load_model("outputs/v1/mildew_detector_cnn_tuned.keras")

### **Evaluate Both Models on the Test Set**

In [ ]:
# Evaluate Original Model
orig_test_loss, orig_test_accuracy = original_model.evaluate(test_set)
print(f"Original CNN Test Accuracy: {orig_test_accuracy:.4f}")
print(f"Original CNN Test Loss: {orig_test_loss:.4f}")

# Evaluate Tuned Model
tuned_test_loss, tuned_test_accuracy = tuned_model.evaluate(test_set)
print(f"Tuned CNN Test Accuracy: {tuned_test_accuracy:.4f}")
print(f"Tuned CNN Test Loss: {tuned_test_loss:.4f}")

### **Compare Accuracy & Loss in a Table**

In [ ]:
import pandas as pd

# Create a comparison table
comparison_df = pd.DataFrame(
    {
        "Model": ["Original CNN", "Tuned CNN"],
        "Test Accuracy": [orig_test_accuracy, tuned_test_accuracy],
        "Test Loss": [orig_test_loss, tuned_test_loss],
    }
)

# Display results
import ace_tools as tools

tools.display_dataframe_to_user(name="Model Comparison", dataframe=comparison_df)

### **Plot Comparison of Accuracy & Loss**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
models = ["Original CNN", "Tuned CNN"]
accuracy_values = [orig_test_accuracy, tuned_test_accuracy]
loss_values = [orig_test_loss, tuned_test_loss]

# Plot Accuracy Comparison
plt.figure(figsize=(8, 5))
plt.bar(models, accuracy_values, color=["blue", "green"])
plt.ylabel("Test Accuracy")
plt.title("Accuracy Comparison: Original vs. Tuned CNN")
plt.ylim(0, 1)
plt.show()

# Plot Loss Comparison
plt.figure(figsize=(8, 5))
plt.bar(models, loss_values, color=["red", "purple"])
plt.ylabel("Test Loss")
plt.title("Loss Comparison: Original vs. Tuned CNN")
plt.show()

**Remarks**
- Tuned CNN has higher test accuracy: Meaning it generalizes better to unseen data.
- Tuned CNN has lower test loss: Meaning it makes more confident and accurate predictions.
- Conclusion: The Tuned CNN is better than the original model! 

---

## **Transfer Learning with MobileNetV2**

---

Implement Transfer Learning using MobileNetV2 to see if it performs even better.

**Why Use MobileNetV2?**

- Pre-trained on ImageNet → Learns faster with fewer data
- Efficient & Lightweight → Works well on limited resources (Codespaces)
- Better Feature Extraction → Captures high-level patterns in leaf images

### **Load MobileNetV2 Pretrained Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


# Function to create a MobileNetV2 model
def create_mobilenet_model(input_shape=(128, 128, 3), num_classes=1):
    """
    Creates a MobileNetV2 model for binary classification.

    - Uses a pre-trained MobileNetV2 backbone.
    - Freezes base layers to retain pre-trained features.
    - Adds a custom classification head.

    Parameters:
        input_shape (tuple): Shape of input images.
        num_classes (int): Number of output classes.

    Returns:
        model (tf.keras Model): Compiled MobileNetV2 model.
    """

    # Load MobileNetV2 with pre-trained weights, excluding the top layers
    base_model = MobileNetV2(
        weights="imagenet", include_top=False, input_shape=input_shape
    )

    # Freeze the base model layers
    base_model.trainable = False

    # Add custom layers for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Reduce feature map size
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)  # Dropout to prevent overfitting
    output_layer = Dense(num_classes, activation="sigmoid")(x)  # Binary classification

    # Create the final model
    model = Model(inputs=base_model.input, outputs=output_layer)

    # Compile the model
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model


# Instantiate the MobileNetV2 model
mobilenet_model = create_mobilenet_model()

# Print model summary
mobilenet_model.summary()

### **Train MobileNetV2**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping callback
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Train the MobileNetV2 model
history_mobilenet = mobilenet_model.fit(
    train_set,
    epochs=10,  
    steps_per_epoch=len(train_set),
    validation_data=validation_set,
    callbacks=[early_stop],
    verbose=1,
)

### **Evaluate MobileNetV2 on Test Data**

In [ ]:
# Evaluate the trained model on the test set
test_loss, test_accuracy = mobilenet_model.evaluate(test_set)

print("\n### MobileNetV2 Model Evaluation ###")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

### **Save MobileNetV2 Model**

In [ ]:
mobilenet_model.save("outputs/v1/mildew_detector_mobilenet.keras")
print("MobileNetV2 Model Saved Successfully!")

### **Compare MobileNetV2 vs. Tuned CNN**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the tuned CNN model
tuned_model = load_model("outputs/v1/mildew_detector_cnn_tuned.keras")
tuned_test_loss, tuned_test_accuracy = tuned_model.evaluate(test_set)

# MobileNetV2 test results (already calculated)
mobilenet_test_loss, mobilenet_test_accuracy = test_loss, test_accuracy

# Create a comparison table
comparison_df = pd.DataFrame(
    {
        "Model": ["Tuned CNN", "MobileNetV2"],
        "Test Accuracy": [tuned_test_accuracy, mobilenet_test_accuracy],
        "Test Loss": [tuned_test_loss, mobilenet_test_loss],
    }
)

# Display the comparison
print("\n### Model Comparison ###")
print(comparison_df)


### **Visualize Model Performance**

In [ ]:
# Plot Accuracy Comparison
plt.figure(figsize=(8, 5))
plt.bar(
    ["Tuned CNN", "MobileNetV2"],
    [tuned_test_accuracy, mobilenet_test_accuracy],
    color=["blue", "green"],
)
plt.ylabel("Test Accuracy")
plt.title("Accuracy Comparison: Tuned CNN vs. MobileNetV2")
plt.ylim(0, 1)
plt.show()

# Plot Loss Comparison
plt.figure(figsize=(8, 5))
plt.bar(
    ["Tuned CNN", "MobileNetV2"],
    [tuned_test_loss, mobilenet_test_loss],
    color=["red", "purple"],
)
plt.ylabel("Test Loss")
plt.title("Loss Comparison: Tuned CNN vs. MobileNetV2")
plt.show()

### **Choose the Best Model & Save for Deployment**

In [ ]:
# Select the best model based on accuracy
if mobilenet_test_accuracy >= tuned_test_accuracy:
    final_model_name = "MobileNetV2"
    final_model_path = "outputs/v1/mildew_detector_mobilenet.keras"
else:
    final_model_name = "Tuned CNN"
    final_model_path = "outputs/v1/mildew_detector_cnn_tuned.keras"

# Load and save the best model as the final deployment model
final_model = load_model(final_model_path)
final_model.save("outputs/v1/final_mildew_detector.keras")

print("\n### Final Model Selected ###")
print(f"Selected Model: {final_model_name}")
print(f"Model saved at: outputs/v1/final_mildew_detector.keras")

---

## **Explainability & Interpretability**

---

### **Grad-CAM Visualization**

Helps us understand which parts of the leaf images the model is focusing on when predicting healthy vs. infected leaves.

**Import Libraries**

In [ ]:
from tensorflow.keras.models import load_model
from tf_explain.core.grad_cam import GradCAM

### **Load the Final Model**

In [ ]:
# Set output directory for saving Grad-CAM visualization
output_dir = "outputs/v1"
os.makedirs(output_dir, exist_ok=True)

# Load the final trained model
final_model = load_model("outputs/v1/final_mildew_detector.keras")

### **Select & Display a Sample Image**

In [ ]:
# Function to select and display a sample image
def load_sample_image(test_set, sample_idx=0):
    """
    Select a sample image from the test set and display it.
    """
    test_images, test_labels = next(iter(test_set))  # Get batch of images
    sample_image = test_images[sample_idx]  # Select one image
    sample_label = test_labels[sample_idx]  # Get its label

    # Display the image
    plt.imshow(sample_image)
    plt.axis("off")
    plt.title(f"Sample Image - {'Healthy' if sample_label == 0 else 'Infected'}")
    plt.show()

    return sample_image, sample_label


# Load and display a sample image from the test set
sample_image, sample_label = load_sample_image(test_set, sample_idx=0)

### **Generate Grad-CAM Heatmap**

In [ ]:
# Function to generate Grad-CAM visualization
def generate_gradcam(model, sample_image, layer_name="Conv_1"):
    """
    Generate and display Grad-CAM heatmap for a given image.

    Parameters:
    - model: Trained model for predictions
    - sample_image: Image to analyze
    - layer_name: Last convolutional layer for Grad-CAM (default: "Conv_1")
    """
    explainer = GradCAM()

    # Ensure correct input shape for the model
    sample_image = np.expand_dims(sample_image, axis=0)

    # Compute Grad-CAM heatmap
    heatmap = explainer.explain(
        (sample_image, None), model, class_index=0, layer_name=layer_name
    )

    # Display the heatmap
    plt.imshow(heatmap)
    plt.axis("off")
    plt.title("Grad-CAM Heatmap")
    plt.show()

    # Save the image
    save_path = os.path.join(output_dir, "grad_cam.png")
    plt.imsave(save_path, heatmap)
    print(f"Grad-CAM visualization saved at: {save_path}")


# Generate and display Grad-CAM heatmap
generate_gradcam(final_model, sample_image)

---

## **Predict on New Images**

---

### **Load & Predict on Sample Image**

Load test images and classify them using the trained model.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the final model
model = load_model("outputs/v1/final_mildew_detector.keras")

# Select an image by specifying its index (pointer)
pointer = 60
label = labels[1]  # Selecting an 'Infected' leaf image

# Load the selected image and resize it
pil_image = image.load_img(
    test_path + "/" + label + "/" + os.listdir(test_path + "/" + label)[pointer],
    target_size=image_shape,
    color_mode="rgb",
)

# Convert the image to an array and normalize it
my_image = image.img_to_array(pil_image) / 255.0
my_image = np.expand_dims(my_image, axis=0)  # Add batch dimension

# Make a prediction
pred_proba = model.predict(my_image)[0, 0]  # Extract prediction probability

# Map indices to class labels
target_map = {v: k for k, v in train_set.class_indices.items()}  # Reverse mapping
pred_class = target_map[int(pred_proba > 0.5)]  # **Fixed: Ensure correct mapping**

# Adjust probability if needed
if pred_class == target_map[0]:
    pred_proba = 1 - pred_proba

# Display results
print(f"Image shape: {pil_image.size}")
print(f"Image mode: {pil_image.mode}")
print(f"Predicted class: {pred_class}")
print(f"Prediction probability: {pred_proba:.4f}")

# Show the image
pil_image


### **Compare Predictions for Multiple Images**

In [ ]:
# Define a list of pointers
pointers = [10, 30, 50, 70]
label = labels[1]  # 'Infected' or 'Healthy'

fig, axes = plt.subplots(1, len(pointers), figsize=(15, 5))

for i, pointer in enumerate(pointers):
    img_list = os.listdir(test_path + "/" + label)

    if pointer >= len(img_list):
        print(f"Skipping pointer {pointer}, index out of range.")
        continue

    img_path = test_path + "/" + label + "/" + img_list[pointer]

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    pred_class = "Healthy" if pred < 0.5 else "Infected"

    # Plot the image and prediction result
    axes[i].imshow(img)
    axes[i].set_title(f"{pred_class}\nProb: {pred:.4f}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

---

## **Conclusion and Next Steps**
---

In this project, we successfully developed a machine learning model to detect **Powdery Mildew on Cherry Leaves** with an efficient and **beginner-friendly approach**.

### **Key Achievements**
- **Baseline CNN Implementation** → Trained a simple CNN model to establish a reference.
- **Transfer Learning with MobileNetV2** → Leveraged pre-trained features for better accuracy and efficiency.
- **Optimized Hyperparameter Tuning** → Applied a simplified tuning approach to balance performance and computation cost.
- **Model Evaluation & Comparison** → Assessed **CNN vs. MobileNetV2** based on accuracy, loss, and efficiency.
- **Explainability with Grad-CAM** → Visualized model focus areas for better interpretability.
- **Final Model Selection** → MobileNetV2 was chosen due to its superior **accuracy and efficiency**.

### **Next Steps: Model Deployment**
The next step is to **deploy the final model** in a **user-friendly application** to allow real-time predictions.

#### **Deployment Plan**
1. **Develop an Interactive Web App** → Users can upload leaf images for classification.
2. **Integrate the MobileNetV2 Model** → Load the trained model into the web application.
3. **Deploy on a Cloud Platform** → Host the app using **Streamlit** for accessibility.

This deployment will enable **real-time detection of powdery mildew**, supporting **efficient and automated plantation management**.

---